# Grundasserspiegel in NÖ

Quellen:

- **Hydrografischer Dienst NÖ:** https://www.noel.gv.at/wasserstand/#/de/Messstellen/Map/Grundwasserspiegel

In [ ]:
import pandas as pd, datetime as dt, numpy as np, re
import requests as req
import matplotlib.pyplot as plt
pd.set_option('display.max_rows', 128)

In [ ]:
stations = pd.read_json("https://www.noel.gv.at/wasserstand/kidata/maplist/MapList.json")
stations = stations[stations.Parameter == "Grundwasserspiegel"]["Parameter	Stationnumber	Stationname	Lat	Long Catchment".split()]   
stations["InfoUrl"] = "https://www.noel.gv.at/wasserstand/kidata/stammdata/" + stations.Stationnumber + ".json"
stations["DownloadUrl"] = "https://www.noel.gv.at/wasserstand/kidata/stationdata/" + stations.Stationnumber + "_Grundwasserspiegel_15Jahre.csv"

In [ ]:
def read_stationinfo(row):
    print(f"Lade Daten der Station {row.Stationnumber}...")
    response = req.get(row.InfoUrl, headers={'Accept': 'application/json'})
    station_info = response.json()
    hoehe_match = re.match(r"([0-9.,]+)", station_info.get("HisGW").get("Messpunkthoehe"))
    row["StationHeight"] = float(hoehe_match.group(0).replace(",", ".")) if hoehe_match else 0
    ngw_match = re.match(r"([0-9.,]+)", station_info.get("HisGW").get("NGW"))
    row["StationNgw"] = float(ngw_match.group(0).replace(",", ".")) if ngw_match else 0
    hgw_match = re.match(r"([0-9.,]+)", station_info.get("HisGW").get("HGW"))
    row["StationHgw"] = float(hgw_match.group(0).replace(",", ".")) if hgw_match else 0
    return row
stations = stations.apply(read_stationinfo, axis=1).convert_dtypes() \
    .rename({"Stationnumber": "StationNumber", "Stationname": "StationName",
             "Lat": "StationLatitude", "Long": "StationLongitude", "Catchment": "StationCatchment"}, axis=1) \
    .astype({"StationNumber": int})

In [ ]:
measures = None
dateparser = lambda values: pd.to_datetime(values, format='%Y-%m-%d %H:%M:%S')
for index, station in stations.iterrows():
    print(f"Lade Messwerte der Station {station.StationNumber} von {station.DownloadUrl}...")
    measure = pd.read_csv(station.DownloadUrl, parse_dates=["Datum"], date_parser=dateparser, 
                          sep=";", header=9, encoding="utf-8", ).convert_dtypes()
    measure["StationNumber"] = station.StationNumber
    measures = measure if measures is None else pd.concat([measures, measure])
measures = measures.set_index("StationNumber")

In [ ]:
# Nur volle Jahre exportieren
year_from = 2009
year_to = 2022

station_measures = stations.drop(["InfoUrl", "DownloadUrl", "Parameter"], axis=1) \
    .join(measures[(measures.Datum.dt.year >= year_from) & (measures.Datum.dt.year <= year_to)], on="StationNumber")
station_measures["GwLevel"] = round(station_measures.Wert - station_measures.StationHeight, 3)
station_measures.to_csv("../grundwasserspiegel_noe.csv", sep=";", encoding="utf-8", index=False)
station_measures.to_csv("../grundwasserspiegel_noe_unicode.csv", sep=";", encoding="utf-16", index=False)

Übersicht über die Anzahl der Messwerte pro Station

In [ ]:
station_measures.groupby(["StationNumber", station_measures.Datum.dt.year]).aggregate(Count=("StationNumber", "size")) \
    .unstack()